In [33]:
import pandas as pd
import numpy as np

from RBO import *
from getRecommendations import *
from dataPrep import *
from sklearn.metrics import *
from scipy.stats import linregress

from sklearn.metrics.pairwise import *

# Read csvs

These are the csv's we cleaned up and combined in step 01. 

In [34]:
%%time
ings = pd.read_csv('data/ings.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prods = pd.read_csv('data/prods.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prod_ing = pd.read_csv('data/prod_ing.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = prod_ing.drop(['diff'], axis = 1)
#prod_ing = prod_ing.sort_values(['id', 'order'])
df = dataPrep(ings, prods, prod_ing)

print('Number of products df: ', len(df))
print('Number of unique ingredients: ', len(ings))
print('Number of total ingredients: ', len(prod_ing))

Number of products df:  15945
Number of unique ingredients:  6899
Number of total ingredients:  446618
Wall time: 6.54 s


In [35]:
%%time
##Long dataframes
df2_A = pd.read_csv('data/prodIng_Wide_01.csv', index_col=0)
df2_B = pd.read_csv('data/prodIng_Wide_02.csv', index_col=0)
df2_C = pd.read_csv('data/prodIng_Wide_03.csv', index_col=0)
frames = [df2_A, df2_B, df2_C]
df2 = pd.concat(frames)
df2 = df2.T

Wall time: 34.2 s


# GET RECOMMENDATIONS


In [36]:
product ='Active Botanical Serum'

##get indices of all products
indices = pd.Series(df.index, index=df['product'])#.drop_duplicates()
##find index of specified product
idx = indices[product]

print('Product:', df.iloc[idx]['product'])
print('Brand:', df.iloc[idx]['brand'])
print('Id:', idx)
print('Number of ingredients:', df.iloc[idx]['ingCount'])
print('Ingredients', df.iloc[idx]['ingList'])

Product: Active Botanical Serum
Brand: Vintner's Daughter
Id: 8083
Number of ingredients: 22
Ingredients ['Vitis Vinifera (Grape) Seed Oil', 'Corylus Avellana (Hazel) Seed Oil', 'Citrus Aurantium Bergamia (Bergamot) Peel Oil', 'Persea Gratissima (Avocado) Oil', 'Calendula Officinalis Extract', 'Rosa Damascena (Rose', 'Daucus Carota Sativa (Carrot) Seed Oil', 'Oenothera Biennis (Evening Primrose) Oil', 'Rosa Rubiginosa Seed Oil', 'Citrus Limon (Lemon) Peel', 'Lavandula Intermedia Flower/Leaf/Stem Extract', 'Medicago Sativa (Alfalfa) Leaf Powder', 'Urtica Dioica (Nettle) Leaf', 'Taraxacum Officinale (Dandelion) Leaf Extract', 'Boswellia Carterii Oil', 'Citrus Aurantium Amara (Bitter Orange) Flower Oil', 'Rosmarinus Officinalis (Rosemary) Leaf Extract', 'Hippophae Rhamnoides Fruit Extract', 'Curcuma Longa (Turmeric) Root Oil', 'Cupressus Sempervirens Leaf Oil', 'Jasminum Sambac (Jasmine) Oil', 'Calophyllum Tacamahaca Seed Oil']


#### Find product id with name

In [15]:
product_find = 'Starting Treatment Essence'
idx_2 = indices[product_find]
#idReturn = row.index.item()

print('Product:', df.iloc[idx_2]['product'])
print('Brand:', df.iloc[idx_2]['brand'])
print('Id:', idx_2)
print('Number of ingredients:', df.iloc[idx_2]['ingCount'])
print('Ingredients', df.iloc[idx_2]['ingList'])

Product: Starting Treatment Essence
Brand: SECRET KEY
Id: 2458
Number of ingredients: 24
Ingredients ['Water', 'Glycerin', 'Galactomyces Ferment Filtrate', 'Butylene Glycol', 'Arbutin', 'Aloe Barbadensis Leaf Juice', 'Oligopeptide-91 rh-Oligopeptide-1', 'Dimethicone', 'Chamomilla Recutita (Matricaria) Flower Extract', 'Centella Asiatica Extract', 'Camellia Sinensis Leaf Extract', 'Physalis Alkekengi Fruit Extract', 'Acanthopanax Senticosus (Eleuthero) Root Extract', 'Ganoderma Lucidum (Mushroom) Extract', 'Panax Ginseng Root', 'Velvet Extract', 'Cordyceps Sinensis Extract', 'Hamamelis Virginiana (Witch Hazel) Extract', 'Oryza Sativa (Rice) Bran Extract', 'Portulaca Oleracea Extract', 'Zanthoxylum Piperitum Fruit Extract', 'Adenosine', 'Phenoxyethanol', 'Disodium EDTA']


In [312]:
idx_2

4350

## Cosine similarity

Reference [here](https://www.datacamp.com/community/tutorials/recommender-systems-python)

### Create dataframe of top ten similar products

In [46]:
%%time
column = 'product'
# Get the index of the product that matches the product name
df['ing#List'] = df['ing#List'].astype(str)
tfidf = TfidfVectorizer(binary = True, stop_words = None)
tfidf_matrix = tfidf.fit_transform(df['ing#List'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

indices = pd.Series(df.index, index=df[column]).drop_duplicates()
idx = indices[product]

# Get the pairwsie similarity scores of all products with that product
sim_scores_all = list(enumerate(cosine_sim[idx]))

# Sort the products based on the similarity scores
sim_scores_sorted = sorted(sim_scores_all, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar products
sim_scores_ten = sim_scores_sorted[1:11]

# Get the product indices
product_indices = [i[0] for i in sim_scores_ten]

# Return the top 10 most similar products and their ingredients
#return df[['product', 'brand','ingList']].iloc[product_indices]

recommended = df[['product', 'brand','ingList']].iloc[product_indices]

Wall time: 29 s


### Show top ten

In [47]:
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
#recommended.to_csv('recommendations/COS.csv')
topTen.reset_index()

,index,product,brand,ingList,commonIng
0,11538,Seriously Soothing Blue Tansy Night Oil,Acure Organics,"[Simmondsia Chinensis (Jojoba) Seed Oil, Capry...",2
1,4004,Rosa Mosqueta Rose Hip Seed Oil,Aubrey Organics,[Rosa Rubiginosa Seed Oil],1
2,11141,Bee Beauty Facial Oil,Elaa Skincare,"[Hydrogenated Jojoba Oil, Rosa Canina (Rosehip...",2
3,11181,Organic Wear Bright Booster Oil Elixir,Physicians Formula,"[Camellia Oleifera Seed Oil, Helianthus Annuus...",3
4,250,Polyphenol C15 Overnight Detox Oil,Caudalie Paris,"[Caprylic/Capric Triglyceride, Vitis Vinifera ...",4
5,3589,Superskin Concentrate,Liz Earle Naturally Active Skin Care,"[Corylus Avellana (Hazel) Seed Oil, Rosa Rubig...",3
6,12610,Elixir Botanical Treatment Serum,Namari Skin,"[Opuntia Ficus-Indica Seed Oil, Argania Spinos...",5
7,12329,Konon Cleansing Cream Concentrate,Namari Skin,"[Anacardium Occidentale (Cashew) Seed Oil, Arg...",3
8,7501,Rose Hips Rose Regeneration Face Creme,Shea Terra Organics,"[Rosa Damascena (Rose, Rosa Rubiginosa Seed Oi...",2
9,11294,Lapis Facial Oil,Herbivore Botanicals,"[Amaranthus Caudatus Seed Oil, Simmondsia Chin...",2


### Find specific product similarity

In [58]:
prod_search_id = 12161
sims = [v for i, v in enumerate(sim_scores_all) if v[0] == prod_search_id]

sim_df = pd.DataFrame(sim_scores_all, columns=['index', 'score']).sort_values(['score'], ascending = False).reset_index().drop(['level_0'], axis = 1)
index_return = sim_df[sim_df['index'] == prod_search_id].index.item() 

list1 = df.loc[idx]['ingList']
list2 = df.loc[prod_search_id]['ingList']

print('Cosine Similarity')
print('---------------------------------------------------------------')
print('Product:', df.iloc[prod_search_id]['product'])
print('Brand:', df.iloc[prod_search_id]['brand'])
print('Similarity score:', sims[0][1])
print('Rank:', index_return)
print('Number of Ingredients', df.iloc[prod_search_id]['ingCount'])
print('Number of Common Ingredients', len(set(list1)&set(list2)))
print('Ingredients', df.iloc[prod_search_id]['ingList'])

Cosine Similarity
---------------------------------------------------------------
Product: Sunshine Oil
Brand: Milk Makeup
Similarity score: 0.08489910975
Rank: 127
Number of Ingredients 10
Number of Common Ingredients 2
Ingredients ['Vitis Vinifera (Grape) Seed Oil', 'Persea Gratissima (Avocado) Oil', 'Citrus Grandis (Grapefruit) Peel Oil', 'Citrus Tangerina (Tangerine) Peel Oil', 'Citrus Nobilis (Mandarin Orange) Peel Oil', 'Citrus Medica Limonum (Lemon) Oil', 'Prunus Armeniaca (Apricot) Kernel Oil', 'Tocopheryl Acetate', 'Lecithin', 'Phenoxyethanol']


## RBO similarity measure

Based on code found [here](https://github.com/dlukes/rbo/blob/master/rbo.py)

### Create dataframe of top ten similar products

In [59]:
%%time

df = dataPrep(ings, prods, prod_ing)
##Set the rbo we want to use
rbo = rbo_min

itemLookup = df.loc[idx]['ing#List']
items = df['ing#List']
sim = [rbo(itemLookup,i, .9) for i in items] ##change method here

sim_scores_all = list(enumerate(sim))

# Sort the products based on the similarity scores
sim_scores_sorted = sorted(sim_scores_all, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar products
sim_scores_ten = sim_scores_sorted[1:11]

# Get the product indices
product_indices = [i[0] for i in sim_scores_ten]

recommended = df[['product', 'brand','ingList']].iloc[product_indices]

Wall time: 14.6 s


### Show top ten

In [60]:
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
#recommended.to_csv('recommendations/RBO.csv')
topTen.reset_index()

,index,product,brand,ingList,commonIng
0,13465,Perfecting Facial Oil,Nuori,"[Vitis Vinifera (Grape) Seed Oil, Prunus Armen...",4
1,12161,Sunshine Oil,Milk Makeup,"[Vitis Vinifera (Grape) Seed Oil, Persea Grati...",2
2,14363,The Oil,Codage,"[Vitis Vinifera (Grape) Seed Oil, Macadamia In...",2
3,11197,Brightening Eye Serum,Three Organics,"[Vitis Vinifera (Grape) Seed Oil, Rosa Canina ...",2
4,11535,Daily Clarifying Treatment Oil,Jouer Cosmetics,"[Vitis Vinifera (Grape) Seed Oil, Simmondsia C...",2
5,3589,Superskin Concentrate,Liz Earle Naturally Active Skin Care,"[Corylus Avellana (Hazel) Seed Oil, Rosa Rubig...",3
6,6673,Seaberry Skin Nutrition Cleansing Oil,fresh,"[Vitis Vinifera (Grape) Seed Oil, Hydrogenated...",2
7,10170,Amazing Face Moringa Cleansing Balm,Emma Hardie,"[Vitis Vinifera (Grape) Seed Oil, Prunus Amygd...",2
8,9787,Rosehip and Argan Antioxidant Serum,White Egret,"[Simmondsia Chinensis (Jojoba) Seed Oil, Vitis...",3
9,402,Love Your Bare Face Hydrating Cleansing Oil,Julep,"[Vitis Vinifera (Grape) Seed Oil, Olea Europae...",2


### Find specific product similarity

In [67]:
prod_search_id = 8720
sims = [v for i, v in enumerate(sim_scores_all) if v[0] == prod_search_id]

sim_df = pd.DataFrame(sim_scores_all, columns=['index', 'score']).sort_values(['score'], ascending = False).reset_index().drop(['level_0'], axis = 1)
index_return = sim_df[sim_df['index'] == prod_search_id].index.item() 

list1 = df.loc[idx]['ingList']
list2 = df.loc[prod_search_id]['ingList']

print('RBO Similarity')
print('---------------------------------------------------------------')
print('Product:', df.iloc[prod_search_id]['product'])
print('Similarity score:', sims[0][1])
print('Rank:', index_return)
#print('Number of Ingredients', df.iloc[prod_search_id]['ingCount'])
#print('Number of Common Ingredients', len(set(list1)&set(list2)))
#print('Ingredients', df.iloc[prod_search_id]['ingList'])

RBO Similarity
---------------------------------------------------------------
Product: Balancing Face Treatment Oil
Similarity score: 0.10199195289490956
Rank: 125


## R value
Reference [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.linregress.html)

In [68]:
%%time
#get linregress similarity and return r-value
sim = [linregress(df2[idx], df2[i])[2] for i in df2]

##collect all similarity scores (r_val)
sim_scores_all = list(enumerate(sim))

# Sort the products based on the similarity scores
sim_scores_sorted = sorted(sim_scores_all, key=lambda x: x[1], reverse=True)
#df['scores'] = [sim_scores[i][1] for i in sim_scores]

# Get the scores of the 10 most similar products
sim_scores_ten = sim_scores_sorted[1:11]

# Get the product indices
product_indices = [i[0] for i in sim_scores_ten]

recommended = df[['product', 'brand','ingList']].iloc[product_indices]

Wall time: 54.3 s


### Return top ten

In [69]:
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
#recommended.to_csv('recommendations/RBO.csv')
topTen.reset_index()

,index,product,brand,ingList,commonIng
0,8720,Balancing Face Treatment Oil,ESPA,"[Cocos Nucifera (Coconut) Oil, Prunus Amygdalu...",5
1,13465,Perfecting Facial Oil,Nuori,"[Vitis Vinifera (Grape) Seed Oil, Prunus Armen...",4
2,9250,Normal Oily Combo Facial Hydrate,Sensatia Botanicals,"[Simmondsia Chinensis (Jojoba) Seed Oil, Prunu...",3
3,13478,The Super Couple Ultra Luxe Anti-Aging Facial ...,Maya Chia,"[Salvia Hispanica Seed Oil, Haematococcus Pluv...",3
4,11894,Replenishing Face Treatment Oil,ESPA,"[Cocos Nucifera (Coconut) Oil, Prunus Amygdalu...",4
5,9916,Soothing Hydration Organic Facial Oil,Madara Cosmetics,"[Geraniol, Persea Gratissima (Avocado) Oil, To...",3
6,9247,cleopatra?s rose facial hydrate,Sensatia Botanicals,"[Simmondsia Chinensis (Jojoba) Seed Oil, Prunu...",3
7,8742,Regenerating Face Treatment Oil,ESPA,"[Simmondsia Chinensis (Jojoba) Seed Oil, Macad...",4
8,12676,Ageless Skin Serum,Prestige Beaute,"[Argania Spinosa (Argan) Oil, Oenothera Bienni...",3
9,250,Polyphenol C15 Overnight Detox Oil,Caudalie Paris,"[Caprylic/Capric Triglyceride, Vitis Vinifera ...",4


### Find specific product similarity

In [76]:
prod_search_id = 8720
sims = [v for i, v in enumerate(sim_scores_all) if v[0] == prod_search_id]

sim_df = pd.DataFrame(sim_scores_all, columns=['index', 'score']).sort_values(['score'], ascending = False).reset_index().drop(['level_0'], axis = 1)
index_return = sim_df[sim_df['index'] == prod_search_id].index.item() 

list1 = df.loc[idx]['ingList']
list2 = df.loc[prod_search_id]['ingList']

print('RVal Similarity')
print('---------------------------------------------------------------')
print('Product:', df.iloc[prod_search_id]['product'])
print('Similarity score:', sims[0][1])
print('Rank:', index_return)
#print('Number of Ingredients', df.iloc[prod_search_id]['ingCount'])
#print('Number of Common Ingredients', len(set(list1)&set(list2)))
#print('Ingredients', df.iloc[prod_search_id]['ingList'])

RVal Similarity
---------------------------------------------------------------
Product: Balancing Face Treatment Oil
Similarity score: 0.198884203579
Rank: 1
